# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

    
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import os
from mmpt import processingmm

 Could not import win32api and/or win32con
 [wrn] tensorboard not available - training not possible
 [wrn] tensorboard not available - training not possible


In [13]:
# set the processing mode to 'online' or 'offline'
# offline sets up processing when the data (input intensities) are stored on a drive
# online sets up processing when the data is loaded in memory (i.e. when processing "live")
data_source = 'online'

# set the parameters to run the script
input_dirs = [r'/home/elea/Documents/HORAO/repositories/processingMM/tests/data_IMP']
calib_dir = r'/media/elea/ssd/NPP/calib/'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
visualization_preset = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
force_reprocess = True

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'no'

# set lu_chipman_backend to 'processing' or 'prediction'
lu_chipman_backend = 'processing'

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550]

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization
workflow_mode = 'default'

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz
save_pdf_figs = False
# NB: processing time without PDDN (with the data on a SSD drive)
# 'no_viz': 0.39s
# 'default', save_pdf_figs False: 1.65s
# 'default', save_pdf_figs True: 2.67s
# 'full', save_pdf_figs False: 3.24s
# 'full', save_pdf_figs True: 5.94s

# define which intrument is being used (currently supported 'IMP', 'IMP_v2')
instrument = 'IMP'

# define if the MM and lu-chipman should be processed using 'torch' or the implementation from Stefano 'c'
mm_computation_backend = 'c'

# set to False if you have no issue with VRAM (usually more than 4Go is enough)
denoise_patch = True

# define if the wavelenghts should be aligned before processing - and used for the computation
align_wls = False

# define if the specular reflections should be removed
remove_reflection = False

binning_factor = 4

mmProcessor = processingmm.MuellerMatrixProcessor(data_source = data_source, wavelengths = wavelengths, input_dirs = input_dirs,
                                calib_dir = calib_dir, visualization_preset = visualization_preset, 
                                PDDN_mode = PDDN_mode, instrument = instrument, remove_reflection = remove_reflection,
                                workflow_mode = workflow_mode, force_reprocess = force_reprocess, save_pdf_figs = save_pdf_figs, 
                                align_wls = align_wls, denoise_patch = denoise_patch, binning_factor = binning_factor,
                                                  mm_computation_backend = mm_computation_backend,
                                lu_chipman_backend = lu_chipman_backend)
print(mmProcessor)

 [info] Switching denoise_patch to False.
 [info] Loading tumor prediction model...
 [info] Loading tumor prediction model done.
 [info] Loading lu-chipman prediction model...
 [info] Loading lu-chipman prediction model done.

Processing parameters:
data_source: online
instrument: IMP
input_dirs: ['/home/elea/Documents/HORAO/repositories/processingMM/tests/data_IMP']
calib_dir: /media/elea/ssd/NPP/calib/
wavelengths: [550]
align_wls: False
PDDN: no
PDDN_models_path: /home/elea/Documents/HORAO/repositories/processingMM/src/mmpt/PDDN_model
denoise_patch: False
visualization_preset: default
workflow_mode: default
save_pdf_figs: False
force_reprocess: True
time_mode: True
remove_reflection: False
binning_factor: 4
mm_computation_backend: c
lu_chipman_backend: processing
folder_eu_time: {}


In [14]:
# functions as previously, from data stored on the drive ('offline' processing)
mmProcessor.batch_process_master()

ValueError: The data_source is set to "online". The function to process the data online is online_processing.

In [15]:
from mmpt import libmpMuelMat

# example with a test case
"""I = libmpMuelMat.read_cod_data_X3D('tests/data/2022-11-02_T_HORAO-59-AF_FR_S1_1/raw_data/550nm/550_Intensite.cod', isRawFlag=1)
A = libmpMuelMat.read_cod_data_X3D('/media/elea/ssd/NPP/calib/2022-11-02_C_1/550nm/550_A.cod', isRawFlag=0)
W = libmpMuelMat.read_cod_data_X3D('/media/elea/ssd/NPP/calib/2022-11-02_C_1/550nm/550_W.cod', isRawFlag=0)"""

I = np.load('/media/elea/ssd/new_instrument/testing_data/2025-03-24_152306_F_FF_HORAO_0005_003/to_process/630_Image_Number_1.npy')
A = np.load('/media/elea/ssd/new_instrument/testing_data/2025-03-24_152306_F_FF_HORAO_0005_003/to_process/A.npy')
W = np.load('/media/elea/ssd/new_instrument/testing_data/2025-03-24_152306_F_FF_HORAO_0005_003/to_process/W.npy')

In [54]:
%%time 
from mmpt.addons import plot_polarimetry

# set up the parameters to get the visualization for
parameters_to_visualize = ['M11', 'linR', 'azimuth', 'totP', 'totD']

# define if the prediction algorithm should be ran
prediction = False

# functions for online processing
import time
start = time.time()
MM, times = mmProcessor.online_processing(I, A, W, binning_factor=4)
if prediction:
    start_prediction = time.time()
    preds, _, _ = mmProcessor.prediction(MM['nM'], MM['Intensity'])
    times['prediction'] = time.time() - start_prediction
start_visualization = time.time()
images = plot_polarimetry.plot_polarimetry_online(mmProcessor, MM, parameters_to_visualize)
times['visualization'] = time.time() - start_visualization
print(times)

print('total time:', time.time() - start)

 [info] Processing without PDDN
processing
Generating the images for visualization:  0.01304173469543457
{'time_processing': {'time_data_loading_GPU': 0, 'time_MM_processing': {'remove_reflection': 0.00177001953125, 'MM_computation': 0.007611274719238281, 'lu_chipman': 0.09312176704406738, 'total': 0.10680389404296875}}, 'time_binning': 0.0875546932220459, 'time_denoising': 2.384185791015625e-07, 'visualization': 0.013142585754394531}
total time: 0.20768189430236816
CPU times: user 1.13 s, sys: 4.46 ms, total: 1.13 s
Wall time: 208 ms


In [10]:
from mmpt.addons.polarpred import save_results
save_results.save_predictions(preds, MM['M11']/np.max(MM['M11']), '/media/elea/ssd', mode = 'normal', model_path = '', path_intensite = None)

 [wrn] The processed /media/elea/ssd/polarimetry/550nm does not exist, skipping overlays.


In [21]:
# compare the results of different backends ('c', 'processing'), ('c', 'prediction'), ('torch', 'processing'), ('torch', 'prediction')
mm_computation_backends = ['c', 'torch']
lu_chipman_backends = ['processing', 'prediction']

mmProcessor.compare_backends(mm_computation_backends, lu_chipman_backends)

 [info] Loading MM model...
 [info] Loading MM model done.

mm_computation_backend: c, lu_chipman_backend: processing
processing without PDDN...
 [wrn] IMPv2 processing of only unprocessed files not implemented yet.


  0%|                                                     | 0/2 [00:00<?, ?it/s]

Processing: /home/elea/Documents/HORAO/repositories/processingMM/tests/data_IMPv2/2025-03-24_152306_F_FF_HORAO_0005_003
 [wrn] clean_up_old_files not implemented yet for IMPv2.
 [info] Test mode: saving to /home/elea/Documents/HORAO/repositories/processingMM/tests/data_IMPv2/2025-03-24_152306_F_FF_HORAO_0005_003/test_backends/630_Image_Number_1/630nm/c_processing


 50%|██████████████████████▌                      | 1/2 [00:00<00:00,  3.12it/s]

processing
 [wrn] No Gaussian fit found for normalization. Returning unnormalized M11.
Processing: /home/elea/Documents/HORAO/repositories/processingMM/tests/data_IMPv2/2025-03-24_152306_F_FF_HORAO_0005_003
 [wrn] clean_up_old_files not implemented yet for IMPv2.


...

Processing: /home/elea/Documents/HORAO/repositories/processingMM/tests/data_IMPv2/2025-03-24_152306_F_FF_HORAO_0005_003
 [wrn] clean_up_old_files not implemented yet for IMPv2.
 [info] Test mode: saving to /home/elea/Documents/HORAO/repositories/processingMM/tests/data_IMPv2/2025-03-24_152306_F_FF_HORAO_0005_003/test_backends/630_Image_Number_1/630nm/torch_prediction


 50%|██████████████████████▌                      | 1/2 [00:00<00:00,  4.40it/s]

 [wrn] No Gaussian fit found for normalization. Returning unnormalized M11.
Processing: /home/elea/Documents/HORAO/repositories/processingMM/tests/data_IMPv2/2025-03-24_152306_F_FF_HORAO_0005_003
 [wrn] clean_up_old_files not implemented yet for IMPv2.


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  4.20it/s]

 [info] Test mode: saving to /home/elea/Documents/HORAO/repositories/processingMM/tests/data_IMPv2/2025-03-24_152306_F_FF_HORAO_0005_003/test_backends/630_Image_Number_2/630nm/torch_prediction
 [wrn] No Gaussian fit found for normalization. Returning unnormalized M11.
processing without PDDN done.
Done.
 [wrn] IMPv2 processing of only unprocessed files not implemented yet.


In [17]:
%%time
from processingmm.addons import predict
mode = 'fast' # mode should be one of 'normal', 'fast' or 'very fast'
times, samples = predict.batch_prediction(mmProcessor.get_parameters(), MM = True, mode = mode)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.76it/s]

torch.Size([1, 11, 388, 516])
/media/elea/ssd/NPP/test/2022-07-06_T_AUTOPSY-BF_FR_S1_1/raw_data/550_Intensite.cod
{
    "preparation": {
        "load_config": 0.006548166275024414,
        "load_models": 0.10728096961975098
    },
    "pre_process": {
        "load_cod": 0.01946234703063965,
        "load_calib": 0.03697371482849121,
        "switch_cuda": 0.00969076156616211,
        "get_tensor": 0.11146759986877441,
        "total": 0.17759990692138672
    },
    "predict": 0.0029888153076171875,
    "save": 0.17706036567687988,
    "total": 0.36295342445373535,
    "per_sample": 0.36295342445373535
}
CPU times: user 1.52 s, sys: 89.8 ms, total: 1.61 s
Wall time: 478 ms


In [6]:
map_gt = {'GM': 1, 'HT': 2, 'IZ': 3, 'TC': 3}
map_pred = {2: 'GM', 0: 'HT', 1: 'TC'}

accuracies = {}

def run_analysis_predictions(samples):

    for sample in samples:
        signal = np.array(Image.open(os.path.join(sample, 'annotation', 'signal.tif')))
        blood = np.array(Image.open(os.path.join(sample, 'annotation', 'blood.tif')))
        signal = np.logical_and(signal, ~blood)

        GT = np.zeros_like(signal, dtype=np.uint8)
        
        GM_WM = np.array(Image.open(os.path.join(sample, 'histology', 'GM_WM.png')))
        signal = np.logical_and(np.sum(GM_WM, axis = 2) !=0, signal)
        GM = np.logical_and(GM_WM[:,:,0] == 153, GM_WM[:,:,1] == 77)
        WM = np.logical_and(GM_WM[:,:,0] == 153, GM_WM[:,:,1] == 153)

        TCC = np.array(Image.open(os.path.join(sample, 'histology', 'TCC.png')))
        HT = np.logical_and(np.logical_and(TCC[:,:,1] == 255, TCC[:,:,2] == 0), WM)
        IZ = np.logical_and(np.logical_and(TCC[:,:,2] == 255, TCC[:,:,1] == 255), WM)
        TC = np.logical_and(TCC[:,:,0] == 255, WM) 
        GT[GM] = 1
        GT[HT] = 2
        GT[np.logical_or(IZ, TC)] = 3
        # GT[TC] = 4

        accuracy = {}
        
        path_predictions = os.path.join(sample, 'predictions')
        for model in os.listdir(path_predictions):
            if os.path.isdir(os.path.join(sample, 'predictions', model)):
                
                preds = np.load(os.path.join(sample, 'predictions', model, 'preds.npy'))
                preds_simplified = preds.argmax(1).squeeze(0)
        
                converted_preds = np.zeros_like(preds_simplified)
                for idx, x in enumerate(preds_simplified):
                    for idy, y in enumerate(x):
                        if signal[idx, idy]:
                            converted_preds[idx, idy] = map_gt[map_pred[y]]
        
                accuracy[model] = np.sum(np.logical_and(converted_preds == GT, signal)) / np.sum(signal)

                TP = np.logical_and(converted_preds == 3, GT == 3)
                Image.fromarray(TP.astype(np.uint8) * 255).save(os.path.join(sample, 'predictions', model, 'TP.png'))

                TN = np.logical_and(converted_preds == 2, GT == 2)
                Image.fromarray(TN.astype(np.uint8) * 255).save(os.path.join(sample, 'predictions', model, 'TN.png'))

                FP = np.logical_and(converted_preds == 3, GT == 2)
                Image.fromarray(FP.astype(np.uint8) * 255).save(os.path.join(sample, 'predictions', model, 'FP.png'))

                FN = np.logical_and(converted_preds == 2, GT == 3)
                Image.fromarray(FN.astype(np.uint8) * 255).save(os.path.join(sample, 'predictions', model, 'FN.png'))

                print(accuracy[model], (np.sum(TP) + np.sum(TN)) / (np.sum(TP) + np.sum(TN) + np.sum(FP) + np.sum(FN)))
        accuracies[sample] = accuracy
        
    return accuracies
    
run_analysis_predictions(samples)

TypeError: expected str, bytes or os.PathLike object, not tuple

In [ ]:
from processingmm.addons import visualization_lines

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
run_all = False
times = visualization_lines.batch_visualization(parameters, run_all)
times

In [26]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths
run_all = False
align_wavelengths.align_wavelengths(parameters['directories'], parameters['PDDN'], run_all, [600], parameters['instrument'])

NotImplementedError: The alignment of wavelengths is not implemented for IMPv2.